# SAE-78: Document Embeddings

Ce notebook implémente la création de vecteurs de documents (reviews) à partir du modèle Word2Vec (SAE-77).

In [ ]:
import pandas as pd
import numpy as np
import sys
import os

# Ajouter le dossier src au path (niveau supérieur car on est dans notebooks/4-text-representation/)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

from src.features import compute_doc_embeddings
from src.text_preprocessing import preprocess_text

## 1. Chargement des données

In [ ]:
# Charger les reviews nettoyées
DATA_PATH = '../../data/cleaned/reviews_clean.parquet'

try:
    df = pd.read_parquet(DATA_PATH)
    print(f"Loaded {len(df)} reviews.")
    
    # POUR LE DEV/DEMO : On prend un échantillon si le dataset est trop gros
    # df = df.head(10000).copy()
    
except FileNotFoundError:
    print(f"File not found: {DATA_PATH}")

## 2. Préparation (Tokenisation)

In [ ]:
# On s'assure d'avoir une colonne de tokens
if 'text_preprocessed' not in df.columns:
    print("Preprocessing text...")
    # Attention: sur tout le dataset, ça peut être long.
    # Idéalement cette étape devrait être faite en amont et sauvegardée.
    df['text_preprocessed'] = df['text'].apply(lambda x: preprocess_text(str(x)))

# Tokenisation simple (split) car text_preprocessed est déjà nettoyé (espaces)
print("Tokenizing...")
df['tokens'] = df['text_preprocessed'].apply(lambda x: x.split() if isinstance(x, str) else [])

## 3. Calcul des Embeddings

In [ ]:
MODEL_PATH = '../../outputs/models/word2vec_yelp.model'

print("Computing document embeddings...")
embeddings = compute_doc_embeddings(df['tokens'], model_path=MODEL_PATH)

print(f"Embeddings shape: {embeddings.shape}")

## 4. Sauvegarde des résultats
On sauvegarde les embeddings pour pouvoir les réutiliser (Clustering, Classif...)

In [ ]:
# Créer un DataFrame avec IDs et Embeddings
# On ne peut pas facilement sauvegarder un array numpy dans parquet directement sans serialisation,
# ou alors on garde 1 colonne par dimension (ex: emb_0, emb_1...).
# Pour simplifier, on stocke souvent sous forme de liste dans parquet.

df_emb = df[['review_id']].copy()
df_emb['doc_embedding'] = list(embeddings)

OUTPUT_PATH = '../../data/cleaned/reviews_embeddings.parquet'

df_emb.to_parquet(OUTPUT_PATH)
print(f"Embeddings saved to {OUTPUT_PATH}")
print(df_emb.head())